In [1]:
import time
import numpy as np
import pandas as pd
import lightgbm as lgb
from functools import partial
import optuna
from sklearn.model_selection import KFold,StratifiedKFold
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA

use_stack = False
new_study = False
do_lda = False
fix_data_skew = False

In [2]:
train_df = pd.read_csv('input/train' + ('_stack' if use_stack else '') + '.csv')
test_df = pd.read_csv('input/test'  + ('_stack' if use_stack else '') + '.csv')

if fix_data_skew:
    trues = train_df.loc[train_df['target'] == 1]
    falses = train_df.loc[train_df['target'] != 1].sample(frac=1)[:len(trues)]
    train_df = pd.concat([trues, falses], ignore_index=True).sample(frac=1)
else:
    train_df = train_df
    
train_df.head()

X_test = test_df.drop('ID_code',axis=1)
X = train_df.drop(['ID_code','target'],axis=1)
y = train_df['target']

In [3]:
if do_lda:    
    lda = LDA(solver='svd', n_components=5, store_covariance=True)
    X_lda = pd.DataFrame(lda.fit_transform(X, y))
    X_test_lda = pd.DataFrame(lda.transform(X_test))
    X["lda"] = X_lda
    X_test["lda"] = X_test_lda

In [4]:
n_splits = 5
folds = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

def objective(X, y, trial):
    score = 0.0

    # based on kernel params
    params = {
        "objective" : "binary",
        "metric" : "auc",
        "boosting": "gbdt",
        "max_depth" : -1,
        "num_leaves" : trial.suggest_int('num_leaves', 14, 16),
        "learning_rate" : 0.01,
        "bagging_freq": trial.suggest_int('bagging_freq', 3, 5),
        "bagging_fraction" : trial.suggest_uniform('bagging_fraction', 0.45, 0.52),
        "feature_fraction" : trial.suggest_uniform('feature_fraction', 0.038, 0.043), 
        "min_data_in_leaf": trial.suggest_int('min_data_in_leaf', 73, 78),
        "min_sum_heassian_in_leaf": trial.suggest_int('min_sum_heassian_in_leaf', 11, 13),
        "tree_learner": "serial", # "serial",
        "num_threads": 6,
        "boost_from_average": "false",
        #"lambda_l1" : 5,
        #"lambda_l2" : 5,
        "bagging_seed" : 13,
        "verbosity" : 1,
        "seed": 42
    }

    for fold_n, (train_index, valid_index) in enumerate(folds.split(X,y)):
        print('Fold', fold_n, 'started at', time.ctime())
        X_train, X_valid = X.iloc[train_index], X.iloc[valid_index]
        y_train, y_valid = y.iloc[train_index], y.iloc[valid_index]
        
        train_data = lgb.Dataset(X_train, label=y_train)
        valid_data = lgb.Dataset(X_valid, label=y_valid)
            
        model = lgb.train(params,train_data,num_boost_round=20000,
                        valid_sets = [train_data, valid_data],verbose_eval=300,early_stopping_rounds=100)
        score += model.best_score['valid_1']['auc'] / n_splits
    print(score)
    return 1.0 - score

In [ ]:
def main():
    f = partial(objective, X, y)
    if new_study:
        study = optuna.create_study(study_name='lgbm_kfold_study', storage='sqlite:///storage.db')
    else:
        study = optuna.Study(study_name='lgbm_kfold_study', storage='sqlite:///storage.db')
    study.optimize(f, n_trials=900)
    print('params:', study.best_params)

if __name__ == '__main__':
    main()

Fold 0 started at Fri Apr  5 16:43:58 2019
Training until validation scores don't improve for 100 rounds.
[300]	training's auc: 0.887699	valid_1's auc: 0.869115
[600]	training's auc: 0.900426	valid_1's auc: 0.880553
[900]	training's auc: 0.906464	valid_1's auc: 0.885999
[1200]	training's auc: 0.910035	valid_1's auc: 0.888683
[1500]	training's auc: 0.91284	valid_1's auc: 0.890565
[1800]	training's auc: 0.915279	valid_1's auc: 0.892147
[2100]	training's auc: 0.917583	valid_1's auc: 0.893412
[2400]	training's auc: 0.919958	valid_1's auc: 0.894538
[2700]	training's auc: 0.922005	valid_1's auc: 0.895456
[3000]	training's auc: 0.923948	valid_1's auc: 0.896131
[3300]	training's auc: 0.925893	valid_1's auc: 0.896694
[3600]	training's auc: 0.927624	valid_1's auc: 0.897174
[3900]	training's auc: 0.929231	valid_1's auc: 0.897718
[4200]	training's auc: 0.930812	valid_1's auc: 0.898117
[4500]	training's auc: 0.932352	valid_1's auc: 0.89842
[4800]	training's auc: 0.933929	valid_1's auc: 0.898767
[51

[I 2019-04-05 16:48:39,098] Finished a trial resulted in value: 0.09991748329535288. Current best value is 0.09976555762908212 with parameters: {'bagging_fraction': 0.48231252804226993, 'bagging_freq': 4, 'feature_fraction': 0.04283121565948458, 'min_data_in_leaf': 76, 'min_sum_heassian_in_leaf': 12, 'num_leaves': 14}.


Fold 0 started at Fri Apr  5 16:48:40 2019
Training until validation scores don't improve for 100 rounds.
[300]	training's auc: 0.887581	valid_1's auc: 0.868852
[600]	training's auc: 0.90031	valid_1's auc: 0.880491
[900]	training's auc: 0.906374	valid_1's auc: 0.886014
[1200]	training's auc: 0.910034	valid_1's auc: 0.888671
[1500]	training's auc: 0.912809	valid_1's auc: 0.890565
[1800]	training's auc: 0.915301	valid_1's auc: 0.892155
[2100]	training's auc: 0.917583	valid_1's auc: 0.893356
[2400]	training's auc: 0.919944	valid_1's auc: 0.894473
[2700]	training's auc: 0.92201	valid_1's auc: 0.895382
[3000]	training's auc: 0.923934	valid_1's auc: 0.895953
[3300]	training's auc: 0.925879	valid_1's auc: 0.896524
[3600]	training's auc: 0.927631	valid_1's auc: 0.897006
[3900]	training's auc: 0.929247	valid_1's auc: 0.8975
[4200]	training's auc: 0.930826	valid_1's auc: 0.897944
[4500]	training's auc: 0.932358	valid_1's auc: 0.898261
[4800]	training's auc: 0.933921	valid_1's auc: 0.898614
[5100

[I 2019-04-05 16:53:17,855] Finished a trial resulted in value: 0.10001303314165488. Current best value is 0.09976555762908212 with parameters: {'bagging_fraction': 0.48231252804226993, 'bagging_freq': 4, 'feature_fraction': 0.04283121565948458, 'min_data_in_leaf': 76, 'min_sum_heassian_in_leaf': 12, 'num_leaves': 14}.


Fold 0 started at Fri Apr  5 16:53:19 2019
Training until validation scores don't improve for 100 rounds.
[300]	training's auc: 0.887835	valid_1's auc: 0.868761
[600]	training's auc: 0.900383	valid_1's auc: 0.880318
[900]	training's auc: 0.906404	valid_1's auc: 0.885898
[1200]	training's auc: 0.910032	valid_1's auc: 0.888607
[1500]	training's auc: 0.912785	valid_1's auc: 0.890496
[1800]	training's auc: 0.915303	valid_1's auc: 0.89217
[2100]	training's auc: 0.917565	valid_1's auc: 0.893469
[2400]	training's auc: 0.919894	valid_1's auc: 0.894426
[2700]	training's auc: 0.922005	valid_1's auc: 0.895423
[3000]	training's auc: 0.923943	valid_1's auc: 0.895997
[3300]	training's auc: 0.92583	valid_1's auc: 0.896461
[3600]	training's auc: 0.927555	valid_1's auc: 0.896939
[3900]	training's auc: 0.929186	valid_1's auc: 0.897344
[4200]	training's auc: 0.930789	valid_1's auc: 0.897785
[4500]	training's auc: 0.932325	valid_1's auc: 0.89814
[4800]	training's auc: 0.933912	valid_1's auc: 0.898548
[510

...

Fold 0 started at Sat Apr  6 16:51:35 2019
Training until validation scores don't improve for 100 rounds.
[300]	training's auc: 0.887722	valid_1's auc: 0.869359
[600]	training's auc: 0.900414	valid_1's auc: 0.880766
[900]	training's auc: 0.906441	valid_1's auc: 0.886298
[1200]	training's auc: 0.910052	valid_1's auc: 0.888832
[1500]	training's auc: 0.912845	valid_1's auc: 0.890823
[1800]	training's auc: 0.915342	valid_1's auc: 0.892411
[2100]	training's auc: 0.917534	valid_1's auc: 0.893693
[2400]	training's auc: 0.919888	valid_1's auc: 0.89461
[2700]	training's auc: 0.922037	valid_1's auc: 0.895558
[3000]	training's auc: 0.923982	valid_1's auc: 0.896134
[3300]	training's auc: 0.92587	valid_1's auc: 0.896673
[3600]	training's auc: 0.927593	valid_1's auc: 0.897024
[3900]	training's auc: 0.929206	valid_1's auc: 0.897414
[4200]	training's auc: 0.930815	valid_1's auc: 0.8978
[4500]	training's auc: 0.932383	valid_1's auc: 0.898079
[4800]	training's auc: 0.933971	valid_1's auc: 0.898471
[5100

[I 2019-04-06 16:56:02,835] Finished a trial resulted in value: 0.10015224028120406. Current best value is 0.09976555762908212 with parameters: {'bagging_fraction': 0.48231252804226993, 'bagging_freq': 4, 'feature_fraction': 0.04283121565948458, 'min_data_in_leaf': 76, 'min_sum_heassian_in_leaf': 12, 'num_leaves': 14}.


Fold 0 started at Sat Apr  6 16:56:05 2019
Training until validation scores don't improve for 100 rounds.
[300]	training's auc: 0.887911	valid_1's auc: 0.869457
[600]	training's auc: 0.900551	valid_1's auc: 0.880364
[900]	training's auc: 0.906579	valid_1's auc: 0.885993
[1200]	training's auc: 0.910111	valid_1's auc: 0.888488
[1500]	training's auc: 0.912913	valid_1's auc: 0.890417
[1800]	training's auc: 0.915357	valid_1's auc: 0.891981
[2100]	training's auc: 0.917678	valid_1's auc: 0.893395
[2400]	training's auc: 0.920043	valid_1's auc: 0.894497
[2700]	training's auc: 0.922103	valid_1's auc: 0.895451
[3000]	training's auc: 0.924033	valid_1's auc: 0.896059
[3300]	training's auc: 0.925938	valid_1's auc: 0.896624
[3600]	training's auc: 0.92764	valid_1's auc: 0.897078
[3900]	training's auc: 0.929236	valid_1's auc: 0.897588
[4200]	training's auc: 0.930842	valid_1's auc: 0.898027
[4500]	training's auc: 0.932368	valid_1's auc: 0.898346
[4800]	training's auc: 0.93394	valid_1's auc: 0.898694
[51

[I 2019-04-06 17:00:43,464] Finished a trial resulted in value: 0.09999267206034768. Current best value is 0.09976555762908212 with parameters: {'bagging_fraction': 0.48231252804226993, 'bagging_freq': 4, 'feature_fraction': 0.04283121565948458, 'min_data_in_leaf': 76, 'min_sum_heassian_in_leaf': 12, 'num_leaves': 14}.


Fold 0 started at Sat Apr  6 17:00:46 2019
Training until validation scores don't improve for 100 rounds.
[300]	training's auc: 0.887853	valid_1's auc: 0.868821
